#### Two Towers Model
* basic example of using Two Tower architecture for recommender systems
* tutorial [here](https://www.tensorflow.org/recommenders/examples/basic_retrieval)

<img src="images/two_tower.gif" alt="drawing" width="700"/>

In [56]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [20]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /Users/darrenshaw/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteUWSUMX/movielens-train.t…

Dataset movielens downloaded and prepared to /Users/darrenshaw/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


#### Example data

In [25]:
for x in movies.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}
{'movie_genres': array([4, 7]),
 'movie_id': b'1457',
 'movie_title': b'Love Is All There Is (1996)'}


2023-01-03 16:45:36.495111: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [26]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


2023-01-03 16:45:50.559697: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


#### Make feature set simple to start
* when only using user_ids and movie_ids it is very similar to a [matrix facorization model](http://yifanhu.net/PUB/cf.pdf)

In [27]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

#### Establish split
* in real-life scenario we would likely split by a time $T$ and have train less than $T$ and validate/test greater than
* here we will just do random

In [28]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

#### Cardinality
* determine how many unique user and movie ids we have so that we can build the proper processing layer space to map id -> int -> embedding
* _This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables._

In [29]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

#### Implement model

##### Query tower
Determine embedding dimensions:
* Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.

The second is to define the model itself. Here, we're going to use Keras preprocessing layers to first convert user ids to integers, and then convert those to user embeddings via an Embedding layer. Note that we use the list of unique user ids we computed earlier as a vocabulary:

In [36]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

###### Note
A simple model like this corresponds exactly to a classic matrix factorization approach. While defining a subclass of tf.keras.Model for this simple model might be overkill, we can easily extend it to an arbitrarily complex model using standard Keras components, as long as we return an embedding_dimension-wide output at the end.

##### Candidate tower


In [37]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

#### Metrics
In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the tfrs.metrics.FactorizedTopK metric. **The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation. (NOTE: THIS IS HOW NEGATIVE SAMPLES ARE BEING CREATED, TFRS IS ALREADY DOING THIS FOR YOU)** 

In our case, that's the movies dataset, converted into embeddings via our movie model:

In [38]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

#### Loss
The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

In [39]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

#### Full model
We can now put it all together into a model. TFRS exposes a base model class (tfrs.models.Model) which streamlines building models: all we need to do is to set up the components in the __init__ method, and implement the compute_loss method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [40]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

#### Instantiate model

In [41]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

shuffle, batch, cache training data

In [42]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

fit

In [43]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 21s 2s/step - factorized_top_k/top_1_categorical_accuracy: 9.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0086 - factorized_top_k/top_10_categorical_accuracy: 0.0191 - factorized_top_k/top_50_categorical_accuracy: 0.0976 - factorized_top_k/top_100_categorical_accuracy: 0.1743 - loss: 69860.5440 - regularization_loss: 0.0000e+00 - total_loss: 69860.5440
Epoch 2/3
10/10 [==============================] - 17s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0027 - factorized_top_k/top_5_categorical_accuracy: 0.0201 - factorized_top_k/top_10_categorical_accuracy: 0.0400 - factorized_top_k/top_50_categorical_accuracy: 0.1689 - factorized_top_k/top_100_categorical_accuracy: 0.2900 - loss: 67489.6243 - regularization_loss: 0.0000e+00 - total_loss: 67489.6243
Epoch 3/3
10/10 [==============================] - 20s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.

##### re: training
As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

Note that, in this example, we evaluate the metrics during training as well as evaluation. Because this can be quite slow with large candidate sets, it may be prudent to turn metric calculation off in training, and only run it in evaluation.

#### Evaluate

In [45]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 6s 723ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top_k/top_10_categorical_accuracy: 0.0197 - factorized_top_k/top_50_categorical_accuracy: 0.1244 - factorized_top_k/top_100_categorical_accuracy: 0.2350 - loss: 31083.1540 - regularization_loss: 0.0000e+00 - total_loss: 31083.1540


{'factorized_top_k/top_1_categorical_accuracy': 0.0012499999720603228,
 'factorized_top_k/top_5_categorical_accuracy': 0.00925000011920929,
 'factorized_top_k/top_10_categorical_accuracy': 0.019700000062584877,
 'factorized_top_k/top_50_categorical_accuracy': 0.12439999729394913,
 'factorized_top_k/top_100_categorical_accuracy': 0.2349500060081482,
 'loss': 28256.123046875,
 'regularization_loss': 0,
 'total_loss': 28256.123046875}

Test set performance is much worse than training performance. This is due to two factors:

1. Our model is likely to perform better on the data that it has seen, simply because it can memorize it. This overfitting phenomenon is especially strong when models have many parameters. It can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data.

2. The model is re-recommending some of users' already watched movies. These known-positive watches can crowd out test movies out of top K recommendations.

The second phenomenon can be tackled by excluding previously seen movies from test recommendations. This approach is relatively common in the recommender systems literature, but we don't follow it in these tutorials. If not recommending past watches is important, we should expect appropriately specified models to learn this behaviour automatically from past user history and contextual information. Additionally, it is often appropriate to recommend the same item multiple times (say, an evergreen TV series or a regularly purchased item).

#### Make predictions
Now that we have a model, we would like to be able to make predictions. We can use the `tfrs.layers.factorized_top_k.BruteForce` layer to do this.

In [52]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
user = str(65)
_, titles = index(tf.constant([user]))
print(f"Recommendations for user {user}: {titles[0, :3]}")

Recommendations for user 65: [b'Fried Green Tomatoes (1991)' b'Quiet Man, The (1952)'
 b'Butch Cassidy and the Sundance Kid (1969)']


#### Export
* we can also export and serve using TensorflowServing

In [53]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["42"])

  print(f"Recommendations: {titles[0][:3]}")

INFO:tensorflow:Assets written to: /var/folders/m8/wy_525cd2c770bj2s6kyfvhr0000gn/T/tmpwtiit1e8/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/wy_525cd2c770bj2s6kyfvhr0000gn/T/tmpwtiit1e8/model/assets


Recommendations: [b'Rudy (1993)' b"Kid in King Arthur's Court, A (1995)"
 b"Preacher's Wife, The (1996)"]


#### Speeding up predictions
We can also export an approximate retrieval index to speed up predictions. This will make it possible to efficiently surface recommendations from sets of tens of millions of candidates.

To do so, we can use the `scann` package. This is an optional dependency of TFRS, and we installed it separately at the beginning of this tutorial by calling !pip install -q scann.

Once installed we can use the TFRS ScaNN layer.

This layer will perform approximate lookups: this makes retrieval slightly less accurate, but orders of magnitude faster on large candidate sets.

##### Note
* ScaNN is an approximate nearest neighbors algo/package from Google Brain
* According to [this issue](https://github.com/google-research/google-research/issues/779) it looks like scan has issues on macOS, so we'll stop here.



In [57]:
# scann_index = tfrs.layers.factorized_top_k.ScaNN(model.user_model)
# scann_index.index_from_dataset(
#   tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
# )

#### Conclusion
In this model, we created a user-movie model. However, for some applications (for example, product detail pages) it's common to perform item-to-item (for example, movie-to-movie or product-to-product) recommendations.

Training models like this would follow the same pattern as shown in this tutorial, but with different training data. Here, we had a user and a movie tower, and used (user, movie) pairs to train them. In an item-to-item model, we would have two item towers (for the query and candidate item), and train the model using (query item, candidate item) pairs. These could be constructed from clicks on product detail pages.